In [1]:
import pandas as pd
import json
import numpy as np

In [3]:
df = pd.read_csv('data/eiti-summary-company-payments.csv')

In [6]:
df = df[df['country'] == "Myanmar"]

In [8]:
df.head()

,created,changed,country,iso3,year,start_date,end_date,company_name,gfs_code,gfs_description,name_of_revenue_stream,currency_code,currency_rate,value_reported,value_reported_as_USD,reporting_url
31969,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1112-E1,"Ordinary taxes on income, profits and capital ...",Revenue Stream: 1112E1 [Ordinary taxes on inco...,MMK,960.5121,4.017128e+10,41822777.0,https://eiti.org/api/v1.0/organisation/31689
31970,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E1,Royalties,Revenue Stream: 1415E1 [Royalties] - Royalties,MMK,960.5121,1.407288e+11,146514378.0,https://eiti.org/api/v1.0/organisation/31689
31971,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E32,Delivered/paid to state-owned enterprise(s),Revenue Stream: 1415E32 [Delivered/paid to sta...,MMK,960.5121,6.139777e+11,639219143.0,https://eiti.org/api/v1.0/organisation/31689
31972,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E32,Delivered/paid to state-owned enterprise(s),Revenue Stream: 1415E32 [Delivered/paid to sta...,MMK,960.5121,1.208082e+11,125774746.0,https://eiti.org/api/v1.0/organisation/31689
31973,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E32,Delivered/paid to state-owned enterprise(s),Revenue Stream: 1415E32 [Delivered/paid to sta...,MMK,960.5121,4.923598e+09,5126014.0,https://eiti.org/api/v1.0/organisation/31689


## Clean data

In [10]:
df["company_name"].unique()

array(['Petronas Carigali Myanmar Inc', 'TOTAL',
       'Daewoo International Corporation', 'PTT Exploration & Production',
       'Goldpetrol Co Ltd', 'MPRL E&P Pte Ltd', 'Nippon Oil', 'UNOCAL',
       'Chinnery Assets Ltd', 'SNOG Pte Ltd', 'Eternal Mining Company Ltd',
       'Kayah State Mining Production Co. (KMPC)',
       'Myanmar Pongpipat Company Limited', 'Nobel Gold',
       'Tha Byu Mining Co., Ltd.', 'Win Myint Mo Industries Co.,Ltd.',
       'DELCO (Developers Enterepreneurs Liaison Construction Organizers Limited',
       'GPS Joint Venture Co., Ltd.', 'Ngwe Yi Pale Mining Co., Ltd',
       'Max Myanmar Manufacturing Co.,',
       'Triple A Cement International Co., Ltd',
       'Myanmar Wanbao Mining Copper', 'Myanmar Yang Tse Copper Ltd',
       'CNMC Nickel Co Ltd. (MCNICO) (Tagaung Taung Nickel project)',
       'Great Genesis Gems Co., Ltd.', 'MYANMAR IMPERIAL JADE',
       'Richest Gems Co., Ltd', 'LINN LETT WIN YADANAR GEMS',
       'EVER WINNER GEMS', 'AUNG HEIN M

In [13]:
companies_info_df = pd.read_csv('data/eiti_summary_companies_cleaned.csv')

In [14]:
companies_info_df.head()

,Legal name,Identification,Company_name_cl,Identification_cl,Commodities
0,Petronas Carigali Myanmar Inc,NC,PETRONAS CARIGALI MYANMAR INC.(BRANCH OFFICE),16FC/2011-2012,Oil & Gas
1,TOTAL,24FC of 1992-1993,TOTAL E & P MYANMAR (YANGON BRANCH),24FC/1992-1993,Oil & Gas
2,Daewoo International Corporation,1 FC / 2002-2003,Posco Daewoo Corporation (Myanmar E&P Office),1FC/2002-2003,Oil & Gas
3,PTT Exploration & Production,84 FC,PTTEP International Ltd. (Yangon Branch),84FC/1995-1996,Oil & Gas
4,Goldpetrol Co Ltd,173 FC,Goldpetrol Co Ltd,173 FC,Oil & Gas


In [16]:
df = pd.merge(df, companies_info_df, left_on='company_name', right_on='Legal name')

In [23]:
df[df['Commodities'] == 'Gems & Jade']['name_of_revenue_stream'].unique()

array(['Revenue Stream: 1415E1 [Royalties] - Royalties',
       'Revenue Stream: 1415E32 [Delivered/paid to state-owned enterprise(s)] - Sale Split',
       'Revenue Stream: 1415E32 [Delivered/paid to state-owned enterprise(s)] - Emporium Fees / Sale Fees',
       'Revenue Stream: 1422E [Administrative fees for government services] - Supervision Fees',
       'Revenue Stream: 1415E5 [Other rent payments] - Other significant payments (&gt; 50,000 USD)'], dtype=object)

In [24]:
df[df['Commodities'] == 'Gems & Jade']['gfs_description'].unique()

array(['Royalties', 'Delivered/paid to state-owned enterprise(s)',
       'Administrative fees for government services', 'Other rent payments'], dtype=object)

In [33]:
df['revenue_stream_short'] = df['name_of_revenue_stream'].str.split(' - ').str[1]


In [39]:
df_gemsjade = df[df['Commodities'] == 'Gems & Jade']

## Remove negative payments for Sankey

In [40]:
df_gemsjade = df_gemsjade[df_gemsjade["value_reported"] >= 0]

## Prepare Source-Target-Value dataframe

In [45]:
links = pd.DataFrame(columns=['source','target','value'])


In [56]:
to_append = df_gemsjade.groupby(['revenue_stream_short'],as_index=False)['value_reported','value_reported_as_USD'].sum()
to_append["target"] = "MGE"
to_append.rename(columns = {'revenue_stream_short':'source', 'value_reported' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['value'], ascending = False)
links = pd.concat([links,to_append])

print(to_append['value'].sum())
links

202525909823.0


,source,target,value,value_reported_as_USD
3,Sale Split,Total,1.341873e+11,139703870.0
2,Royalties,Total,5.893137e+10,61354119.0
0,Emporium Fees / Sale Fees,Total,6.127875e+09,6379800.0
4,Supervision Fees,Total,3.189849e+09,3320985.0
1,"Other significant payments (&gt; 50,000 USD)",Total,8.955549e+07,93237.0
65,"Great Genesis Gems Co., Ltd.",Sale Split,1.500496e+10,15621836.0
80,Wai Aung Ka Bar,Sale Split,1.339731e+10,13948088.0
74,"Richest Gems Co., Ltd",Sale Split,1.210528e+10,12602939.0
83,Ya Zar Htar Ni,Sale Split,1.034887e+10,10774326.0
79,THI RAW MANI GEMS,Sale Split,8.613594e+09,8967710.0


In [57]:
to_append = df_gemsjade.groupby(['revenue_stream_short','company_name'],as_index=False)['value_reported','value_reported_as_USD'].sum()
to_append.rename(columns = {'company_name':'source','revenue_stream_short':'target', 'value_reported' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['value'], ascending = False)
links = pd.concat([links,to_append])

print(to_append['value'].sum())
links

202525909823.0


,source,target,value,value_reported_as_USD
3,Sale Split,Total,1.341873e+11,139703870.0
2,Royalties,Total,5.893137e+10,61354119.0
0,Emporium Fees / Sale Fees,Total,6.127875e+09,6379800.0
4,Supervision Fees,Total,3.189849e+09,3320985.0
1,"Other significant payments (&gt; 50,000 USD)",Total,8.955549e+07,93237.0
65,"Great Genesis Gems Co., Ltd.",Sale Split,1.500496e+10,15621836.0
80,Wai Aung Ka Bar,Sale Split,1.339731e+10,13948088.0
74,"Richest Gems Co., Ltd",Sale Split,1.210528e+10,12602939.0
83,Ya Zar Htar Ni,Sale Split,1.034887e+10,10774326.0
79,THI RAW MANI GEMS,Sale Split,8.613594e+09,8967710.0


In [58]:
#to_append = df.groupby(['Payment Type','Reporting Company'],as_index=False)['Value (USD)'].sum()
#to_append.rename(columns = {'Payment Type':'source','Reporting Company':'target', 'Value (USD)' : 'value'}, inplace = True)
#to_append = to_append.sort_values(by=['value'], ascending = False)
#links = pd.concat([links,to_append])

#print(to_append['value'].sum())
#links

In [59]:
unique_list = pd.concat([links['source'], links['target']]).unique()
replace_dict = {k: v for v, k in enumerate(unique_list)}


In [60]:
links_replaced = links.replace({"source": replace_dict,"target": replace_dict})

In [61]:
links_replaced

,source,target,value,value_reported_as_USD
3,0,35,1.341873e+11,139703870.0
2,1,35,5.893137e+10,61354119.0
0,2,35,6.127875e+09,6379800.0
4,3,35,3.189849e+09,3320985.0
1,4,35,8.955549e+07,93237.0
65,5,0,1.500496e+10,15621836.0
80,6,0,1.339731e+10,13948088.0
74,7,0,1.210528e+10,12602939.0
83,8,0,1.034887e+10,10774326.0
79,9,0,8.613594e+09,8967710.0


In [62]:
nodes = pd.DataFrame(unique_list)
nodes.rename(columns = {0:'name'}, inplace = True)

In [63]:
nodes_json= pd.DataFrame(nodes).to_json(orient='records')
nodes_json 

'[{"name":"Sale Split"},{"name":"Royalties"},{"name":"Emporium Fees \\/ Sale Fees"},{"name":"Supervision Fees"},{"name":"Other significant payments (&gt; 50,000 USD)"},{"name":"Great Genesis Gems Co., Ltd."},{"name":"Wai Aung Ka Bar"},{"name":"Richest Gems Co., Ltd"},{"name":"Ya Zar Htar Ni"},{"name":"THI RAW MANI GEMS"},{"name":"Kyauk Sein Na Gar"},{"name":"Sein Lone Taung Tan"},{"name":"Jade Mountain Gems"},{"name":"Myanmar Si Thu"},{"name":"111"},{"name":"SHINING STAR LIGHT GEMS"},{"name":"LINN LETT WIN YADANAR GEMS"},{"name":"Ya Da Nar Taung Tan"},{"name":"Aye Yar Kyauk Sein"},{"name":"Xie Family"},{"name":"EVER WINNER GEMS"},{"name":"MYAT YAMON GEMS"},{"name":"Kyaik International"},{"name":"MYANMAR IMPERIAL JADE"},{"name":"Khin Zaw Aung and Brothers"},{"name":"NAY LA PWINT GEMS"},{"name":"WINN LEI YADANA"},{"name":"AUNG HEIN MIN GEMS"},{"name":"SHWE YWET HLWAR GEMS"},{"name":"YADANAR YAUNG CHI GEMS"},{"name":"A Myo Thar Kyi Pwar Toe Tat Yay"},{"name":"WAI FAMILY GEMS"},{"name":"AY

In [64]:
links_json= pd.DataFrame(links_replaced).to_json(orient='records')
links_json 

'[{"source":0,"target":35,"value":134187256546.5733337402,"value_reported_as_USD":139703870.0},{"source":1,"target":35,"value":58931373949.1452789307,"value_reported_as_USD":61354119.0},{"source":2,"target":35,"value":6127875066.9551591873,"value_reported_as_USD":6379800.0},{"source":3,"target":35,"value":3189848769.4279022217,"value_reported_as_USD":3320985.0},{"source":4,"target":35,"value":89555490.4148460031,"value_reported_as_USD":93237.0},{"source":5,"target":0,"value":15004962403.7789993286,"value_reported_as_USD":15621836.0},{"source":6,"target":0,"value":13397307541.9280014038,"value_reported_as_USD":13948088.0},{"source":7,"target":0,"value":12105275122.6339988708,"value_reported_as_USD":12602939.0},{"source":8,"target":0,"value":10348870278.4599990845,"value_reported_as_USD":10774326.0},{"source":9,"target":0,"value":8613593872.3738002777,"value_reported_as_USD":8967710.0},{"source":10,"target":0,"value":6952791229.9081001282,"value_reported_as_USD":7238630.0},{"source":11,"

In [65]:
data = { 'links' : json.loads(links_json), 'nodes' : json.loads(nodes_json) }
data_json = json.dumps(data)
data_json = data_json.replace("\\","")
#print(data_json)
#with open('sankey_data.json', 'w') as outfile:
#    json.dump(data_json, outfile)

text_file = open("sankey_data.json", "w")
text_file.write(data_json)
text_file.close()